In [0]:
CREATE OR REFRESH STREAMING TABLE bronzee_pit_stops
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
SELECT *,
       _metadata.file_path AS input_file_path,
       CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/ishwariformulabox/schemalanding/volume2/pit_stops.csv',
  'csv',
  map('cloudFiles.inferColumnTypes', 'true')
);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_pitstops (
  CONSTRAINT valid_race_id EXPECT (raceId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_driver_id EXPECT (driverId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_stop EXPECT (stop IS NOT NULL AND stop >= 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_lap EXPECT (lap IS NOT NULL AND lap > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_time EXPECT (time IS NOT NULL AND time REGEXP '^\\d{2}:\\d{2}:\\d{2}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_duration EXPECT (duration IS NOT NULL AND duration >= 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_milliseconds EXPECT (milliseconds IS NOT NULL AND milliseconds >= 0) ON VIOLATION DROP ROW
)
COMMENT 'Cleaned lap time data'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT raceId,
       driverId,
       stop,
       lap,
       time,
       duration,
       milliseconds
FROM STREAM(bronzee_pit_stops);
